<a href="https://colab.research.google.com/github/Jydra/variable-stroke-linkage/blob/main/Crank_Symbolic_Comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from os.path import basename, exists
import math
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download("https://raw.githubusercontent.com/Jydra/sympy-interface/main/interface.py")

Downloaded interface.py


In [5]:
##SYMBOLIC INPUTS
from interface import *
from sympy import *
theta1, theta2, theta3, theta4, x1, x2, x3, x4, y1, y2, y3, y4, l1, l2, l3, l4, Rx, Ry, Cx, Cy, theta, u, phi = symbols(r'\theta_1 \theta_2 \theta_3 \theta_4 x_1 x_2 x_3 x_4 y_1 y_2 y_3 y_4 l_1 l_2 l_3 l_4 R_x R_y C_x C_y \theta u phi')
#link 1 (l = l1, ang = theta1, nodes 1,2): input link
#link 2 (l = l2, ang = theta2, nodes 2,3): connecting link
#link 3 (l = l3, ang = theta3, nodes 3,4): adjustment link
#link 4 (l = l4, ang = theta4, nodes 3,5): output link
#R (Rx, Ry): fixed link 3 position along adjustment slot
#C (Cx, Cy): center of adjustment slot radius
#theta: parameterized driving link
#phi: Angle from C to R

#@title ## Solver Output Control
verbose_mode = True #@param {type:"boolean"}
verbose_display_precision = 5 #@param {type:"integer"}

#Dictionaries for Values, Relations, and Temporary Substitutions
#Keys have valid LaTeX formatting so that they an easily be formatted when passed directly from exec_op() into verbose()
#The keys for variables should be identical to the formatting of their counterparts in the symbol declarations (line 2) if they exist
#I am lazy and do not want to add a separate formatting entry into the op structure
vset = dict()
rln = dict()
tmp = dict()

#Set Values
vset[x1] = Eq(x1, 0)
vset[y1] = Eq(y1, 0)
vset[Cx] = Eq(Cx, 0.11698)
vset[Cy] = Eq(Cy, 0.2378)
vset[l1] = Eq(l1, 0.045)
vset[l2] = Eq(l2, 0.2200154)
vset[l3] = Eq(l3, 0.2)

#Relations
rln[theta1] = Eq(theta1, theta)
rln[Rx] = Eq(Rx, Cx+l3*cos(phi))
rln[Ry] = Eq(Ry, Cy+l3*sin(phi))
rln[x2] = Eq(x2, l1*cos(theta1))
rln[y2] = Eq(y2, l1*sin(theta1))
rln[x3] = Eq(x3, x4 + l3*cos(theta3))
rln[y3] = Eq(y3, y4 + l3*sin(theta3))
rln[x4] = Eq(x4, Rx)
rln[y4] = Eq(y4, Ry)
rln[l2**2] = Eq(l2**2, (x2 - x3)**2 + (y2 - y3)**2)

#Temporary substitutions
tmp[x1] = Eq(x1, Rx**2 + Ry**2 + l1**2-l2**2+l3**2-2*Rx*l1*cos(theta)-2*Ry*l1*sin(theta))
tmp[x2] = Eq(x2, 2*Rx*l3-2*l1*l3*cos(theta))
tmp[x3] = Eq(x3, 2*Ry*l3-2*l1*l3*sin(theta))

#Initial equation
eq = rln[l2**2]

#All operations
op = [
    ["init"],                                             #Begin
    ["var_expand",rln,[x2, y2, x3, y3]],      #expand relations from x2, y2, x3, y3
    ["var_expand",rln,[theta1, x4, y4]],       #expand relations from theta1, x4, y4
    ["expand"],                                           #expand equation
    ["to_lhs"],                                           #move all terms to lhs
    ["pythag_id"],                                        #apply pythagorean identity
    ["expand"],                                           #expand equation
    ["collect_lhs",sin(theta3)],                          #collect all sin(theta3) on lhs
    ["collect_lhs",cos(theta3)],                          #collect all cos(theta3) on lhs
    ["var_collect",tmp,[x1, x2, x3]],            #collect temp into x1, x2, x3
    ["solve",theta3, 1],                                  #solve equation for theta3, select 2nd solution
    ["var_expand",tmp,[x1, x2, x3]],             #expand temp from x1, x2, x3
    ["var_expand",rln,[Rx, Ry]],                    #expand relations from Rx, Ry
    ["var_set",vset,[Cx, Cy, l1, l2, l3]]  #expand values from Cx, Cy, l1, l2, l3
]

for i in range(len(op)):
  eq = exec_op(i,op,eq, verbose_mode, verbose_display_precision)

#Lambdify
t3 = lambdify([phi, theta],eq.rhs)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [25]:
##VISUALIZATION
%matplotlib inline
from ipywidgets import Layout, interact, interactive, FloatSlider, IntSlider
import ipywidgets as widgets

def f2(phi, theta):
  plt.figure(figsize=(6, 6))
  x1 = 0
  y1 = 0
  x2 = 0.045*math.cos(math.radians(theta))
  y2 = 0.045*math.sin(math.radians(theta))
  x4 = 0.11698 + 0.2*math.cos(math.radians(phi))
  y4 = 0.2378 + 0.2*math.sin(math.radians(phi))
  x3 = x4 + 0.2*math.cos((t3(math.radians(phi),math.radians(theta)) % math.radians(360)))
  y3 = y4 + 0.2*math.sin((t3(math.radians(phi),math.radians(theta)) % math.radians(360)))

  l1x, l1y = [x1, x2], [y1, y2] 
  l2x, l2y = [x2, x3], [y2, y3] 
  l3x, l3y = [x3, x4], [y3, y4] 
  l4x, l4y = [x3,(x3+(0.2**2-(0.215-y3)**2)**0.5)], [y3,0.215]
  #Optional 80mm ruler
  #cx, cy = [0.23,0.31],[0.215, 0.215]
  #plt.plot(cx, cy)
  plt.plot(l1x, l1y, color = 'black', lw = 3)
  plt.plot(l2x, l2y, color = 'black', lw = 3)
  plt.plot(l3x, l3y, color = 'black', lw = 3)
  plt.plot(l4x, l4y, color = 'black', lw = 3)
  plt.xlim(-0.05, 0.35)
  plt.ylim(-0.05, 0.35)


  ##Helper Elements
  #C
  plt.plot(0.11698, 0.2378, marker = "o", color = 'red')
  plt.gca().annotate('Slot R Center', xy=(0.055, 0.27), fontsize=10, rotation = 0)
  #C to node 4
  plt.plot([0.11698, x4], [0.2378, y4], color = 'green', lw = 1)
  #horizontal at C
  plt.plot([0.11698, 0.30], [0.2378, 0.2378], color = 'green', lw = 1)
  #horizontal at node 1
  plt.plot([0,0.045],[0,0], color = 'green', lw = 1)
  #Slot
  arc1_angles = np.linspace(math.radians(-64.88), math.radians(-25.12), 20)
  arc1_xs = 0.11698 + 0.2 * np.cos(arc1_angles)
  arc1_ys = 0.2378 + 0.2 * np.sin(arc1_angles)    
  plt.plot(arc1_xs, arc1_ys, color = 'red', lw = 1)
  plt.gca().annotate('Slot', xy=(0.26, 0.07), fontsize=10, rotation = 45)
  #phi
  arc2_angles = np.linspace(math.radians(phi), math.radians(0), 20)
  arc2_xs = 0.11698 + 0.01 * np.cos(arc2_angles)
  arc2_ys = 0.2378 + 0.01 * np.sin(arc2_angles)
  plt.plot(arc2_xs, arc2_ys, color = 'blue', lw = 1)
  plt.gca().annotate('ϕ', xy=(0.12, 0.245), fontsize=10, rotation = 0)
  #theta
  arc3_angles = np.linspace(math.radians(0), math.radians(theta), 20)
  arc3_xs = 0.0075 * np.cos(arc3_angles)
  arc3_ys = 0.0075 * np.sin(arc3_angles)
  plt.plot(arc3_xs, arc3_ys, color = 'blue', lw = 1)
  plt.gca().annotate('θ', xy=(0.005, 0.015), fontsize=10, rotation = 0)

  plt.gca().set_aspect("equal")
  plt.show()

interactive_plot = interactive(f2, phi=FloatSlider(value = -64.88, min = -64.88, max = -25.12, step = 0.01, layout=Layout(width='600px')), theta=IntSlider(min = 0, max = 359, step = 1, layout=Layout(width='600px')))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=-64.88, description='phi', layout=Layout(width='600px'), max=-25.12, m…